In [1]:
cd ../..

c:\Users\Josu\Documents\Workspace\Human-Benchmark\src\server\model


In [ ]:
! del /q models\logs\*
! for /d %x in (models\logs\*) do @rd /get_model

In [ ]:
%load_ext tensorboard

In [2]:
import glob
from datetime import datetime
from os import path

import chess
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from tensorflow.keras import callbacks, models, utils

# from models import get_model

In [3]:
XX, yy_1, yy_2 = [], [], []
for file in glob.glob("data/npz/*/*.npz"):
    with np.load(file) as f:
        XX.append(f["X"]), yy_1.append(f["y_1"]), yy_2.append(f["y_2"])
X, y_1, y_2 = np.concatenate(XX), np.concatenate(yy_1), np.concatenate(yy_2)

In [ ]:
plt.title("Policy Head")
plt.xlabel("Index")
plt.ylabel("Count")
_ = plt.hist(y_1, bins=1858)

In [ ]:
plt.title("Value Head")
plt.xlabel("Score")
plt.ylabel("Density")
_, x, _ = plt.hist(y_2, bins=100, density=True)  
_ = plt.plot(x,  stats.gaussian_kde(y_2)(x))

In [6]:
model = get_model()
model.summary()

________________________
add_6 (Add)                     (None, 8, 8, 128)    0           multiply_3[0][0]                 
                                                                 lambda_3[0][1]                   
__________________________________________________________________________________________________
add_7 (Add)                     (None, 8, 8, 128)    0           add_6[0][0]                      
                                                                 re_lu_6[0][0]                    
__________________________________________________________________________________________________
re_lu_8 (ReLU)                  (None, 8, 8, 128)    0           add_7[0][0]                      
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 8, 8, 128)    147584      re_lu_8[0][0]                    
____________________________________________________________________________________

In [ ]:
utils.plot_model(
    model,
    to_file=f"models/png/model.png",
    show_shapes=True,
    show_layer_names=False
)

In [ ]:
callback = (
    callbacks.TensorBoard(
        log_dir=f"models/logs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    ),
    callbacks.ModelCheckpoint(
        f"models/ser/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    )
)

In [7]:
model.compile(
    optimizer="adam",
    loss=("sparse_categorical_crossentropy", "mse")
)

In [8]:
_ = model.fit(
    x=X, y=(y_1, y_2),
    batch_size=256,
    epochs=1,.
    # callbacks=callback,
    validation_split=.2
)

6266/6266 [==============================] - 27690s 4s/step - loss: 2.9538 - policy_loss: 2.8329 - value_loss: 0.1209 - val_loss: 2.4352 - val_policy_loss: 2.3117 - val_value_loss: 0.1235


In [17]:
model.save("models/ser/v1/model.h5")

In [ ]:
%tensorboard --logdir models/logs

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (Activation, Conv2D, Dense, Flatten,
                                     GlobalAveragePooling2D, Input, Lambda,
                                     ReLU, Reshape, add, multiply)


def policy_head(inputs: tf.Tensor, /) -> tf.Tensor:
    x = conv_block(inputs, filters=32)
    x = Flatten()(x)
    return Dense(1858, activation="softmax", name="policy")(x)


def value_head(inputs: tf.Tensor, /) -> tf.Tensor:
    x = conv_block(inputs, filters=32)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    return Dense(1, activation="tanh", name="value")(x)


def squeeze_and_exite(inputs: tf.Tensor, /, units: int) -> tf.Tensor:
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(32, activation="relu")(x)
    x = Dense(2 * units)(x)
    x = Reshape((2, units))(x)
    W, B = Lambda(tf.unstack, arguments={"axis": 1})(x)
    Z = Activation("sigmoid")(W)
    return add((multiply((Z, inputs)), B))


def conv_block(inputs: tf.Tensor, /, filters: int, *, units: int = None, skip_connection: tf.Tensor = None) -> tf.Tensor:
    x = Conv2D(filters, (3, 3), padding="same")(inputs)
    if units is not None:
        x = squeeze_and_exite(x, units=units)
    if skip_connection is not None:
        x = add((x, skip_connection))
    return ReLU()(x)


def residual_block(inputs: tf.Tensor, /, filters: int) -> tf.Tensor:
    x = conv_block(inputs, filters=filters)
    return conv_block(x, filters=filters, units=128, skip_connection=inputs)


def get_model() -> keras.Model:
    inputs = Input(shape=(8, 8, 6))
    x = conv_block(inputs, filters=128)
    for _ in range(10):
        x = residual_block(x, filters=128)
    outputs = policy_head(x), value_head(x)
    return keras.Model(inputs=inputs, outputs=outputs)